In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [ ]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

In [ ]:
# 模型构造


# 层和块
# 首先，我们回顾一下多层感知机
import torch
from torch import nn
from torch.nn import functional as F # 定义了一下没有定义参数的函数

# 简单的单层神经网络 线性层+relu+线性层
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
# nn.Sequential类定义了一个特殊的module，在pytorch里面module是一个很重要的概念

# 2是批量大小，20是输入维度
# 这里相当于是多批次中，每一批的批量大小是2，特征输入维度是20
X = torch.rand(2, 20)
net(X)

tensor([[ 0.2056,  0.2494, -0.0433,  0.0193,  0.2222,  0.0436,  0.0318, -0.0826,
         -0.1385, -0.1381],
        [-0.0225,  0.1790,  0.0326,  0.0707,  0.3704,  0.0487, -0.0181, -0.0439,
         -0.1270,  0.0567]], grad_fn=<AddmmBackward0>)

In [ ]:
# module可以被认为是任何一个层和神经网络都是module的一个子类
# 这里我们定义一个mlp 他是nn.Module的一个子类
# 所有的Module有两个比较重要的函数，一个是init一个是forward
# super().__init__()调用父类函数把初始化参数都设好
# 接下来定义两个全连接层
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播，即如何根据输入x返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义
        return self.out(F.relu(self.hidden(X)))

In [ ]:
# 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层
net = MLP() # 实例化这个类
net(X)

tensor([[ 0.1091, -0.0735, -0.3203,  0.0730, -0.0719, -0.0036, -0.0439,  0.1774,
         -0.1106,  0.0956],
        [ 0.1648,  0.0190, -0.1834,  0.0334, -0.1987, -0.0961,  0.0122,  0.3505,
         -0.1035,  0.0635]], grad_fn=<AddmmBackward0>)

In [ ]:
# 顺序块
# 这里实现了跟nn.Sequential类几乎一样的功能
# 定义一个nn.Module的子类，接收了一个list of input arguments
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            # _modules是一个特殊的容器，里面放的是我需要的层
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [ ]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0269,  0.2151,  0.0648,  0.1236,  0.1104,  0.2016, -0.0252, -0.0794,
          0.2110,  0.0042],
        [ 0.0653,  0.2544, -0.2109, -0.0566,  0.1574,  0.1763,  0.0491, -0.1982,
          0.2015, -0.0073]], grad_fn=<AddmmBackward0>)

In [ ]:
# 当Sequential类不能满足我们的需求的时候，我们通过自定义的能做大量计算
# 我们可以做很多灵活的方法来做正向传播
# 反向计算都是自动求导

# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 用torch.rand直接生成了rand_weight，这个weight是不参与训练的
        # requires_grad=False 不参加训练因为他不会计算梯度
        # 不计算梯度的随机权重参数。因此其在训练期间保存不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    # forward里可以做任何事情
    def forward(self, X):
        X = self.linear(X)
        # 手写用rand_weight和X做矩阵乘法，+1是我们假设的偏移，再做激活函数
        # 然后又回过去调用一下线性类
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1924, grad_fn=<SumBackward0>)

In [ ]:
# 混合搭配各种组合快的方法 -- 嵌套使用
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 首先定义net是一个Sequential的类和一个线性类
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

# 对于Sequential来说输入可以是任何nn.Module的子类
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0747, grad_fn=<SumBackward0>)

In [ ]:
# 参数管理
# 假设我们已经定义好我们的类了，参数怎么样去访问
# 我们首先关注具有单隐藏层的多层感知机

import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.5666],
        [-0.7156]], grad_fn=<AddmmBackward0>)

In [ ]:
# 参数访问
# 我们要干的事情是把每一层里的权重拿出来
# net是nn.Sequential，Sequential可以简单的理解为python里的一个list
# net[2]拿到的就是nn.Linear(8, 1)最后的输出层
# state_dict 从自动机的角度来讲，他的权重就是他的状态
# 他的dict就是_modules这样的东西，是OrderedDict
# 这里就是最后一层的参数
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.3364, -0.1354, -0.2344, -0.0526, -0.3515, -0.0559, -0.2489, -0.0597]])), ('bias', tensor([-0.2108]))])


In [ ]:
# 目标参数
# Parameter定义的是一个可以优化的参数
# 通过.data访问真正的值，.grad访问梯度
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2108], requires_grad=True)
tensor([-0.2108])


In [ ]:
net[2].weight.grad == None

True

In [ ]:
# 一次性访问所有参数
# 1就是relu，relu没有参数的，所以拿不出来
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [ ]:
# 访问最后一层的偏移
net.state_dict()['2.bias'].data

tensor([-0.2108])

In [ ]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # add_module和之前唯一的区别是可以穿个字符串的名字，而不是直接1234了
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2496],
        [0.2496]], grad_fn=<AddmmBackward0>)

In [ ]:
# 我们已经设计了网络，让我们看看它是如何组织的
# 三个sequential的嵌套，这里是比较简单的网络了
# 复杂的网络也是建议模块化
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [ ]:
rgnet[0][1][0].bias.data

tensor([ 0.2166,  0.2875,  0.3422,  0.2011,  0.2665, -0.1356,  0.2322, -0.4574])

In [ ]:
# 内置初始化
# m就是一个Module，如果Module是线性的/全连接层
def init_normal(m):
    if type(m) == nn.Linear:
        # normal_ 下划线在后面表示是一个替换函数，不会返回值
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

# apply函数意思是对所以net里面的那些layer，一个一个做for loop
# 对net里所有module调用init_normal这个函数，把module做为参数传入
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0139,  0.0003, -0.0014, -0.0150]), tensor(0.))

In [ ]:
# 初始成constant
def init_constant(m):
    if type(m) == nn.Linear:
        # weight初始为constant 1
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]
# api角度可以做，但是算法角度不能把weight全部初始化成常数

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
# 对某些块应用不同的初始化方法
# 对不同层apply不一样的东西
# xavier函数，用uniform distribution做初始化
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

# 第一个线性全连接层用Xavier来初始化
net[0].apply(init_xavier)
# 最后一个全连接层用init_42初始化
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

# 因为每个层都是一个module，所以可以对任何一个层调用单独的初始化函数

tensor([ 0.5273,  0.2613,  0.2235, -0.0814])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",
              *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 保留绝对值大于5的权重，不是的就设为零
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, 9.4622, -0.0000, 0.0000],
        [0.0000, -0.0000, 0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [ ]:
# 简单暴力的方法
# 所以值加1，第一个值等于42
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.4622,  1.0000,  1.0000])

In [ ]:
# 参数绑定
# 我们想要在一些层里面share parameter -- 参数绑定
# 我们需要给共享层一个名称，以便可以引用它的参数
# 第二个和第三个隐藏层是shared权重，第一个和最后一个是自己的
# 理论上来说不管怎么更新这个network，他的第二个和第三个层都是一样的
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
# 检查参数是否相同
# 记得第3是relu，所以第二个共享层就是第4
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 改了第一个shared的weight的话，第二个shared的weight也是更改了，
# 因为它们指向了同样的类的实例，这就是如何在不同的网络之间共享权重的一个方法
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
